# Time Series

## Imports

In [27]:
import pandas as pd
import plotly
from tqdm import tqdm
from bubbly.bubbly import make_grid, set_layout, get_trace

## Import File

In [45]:
dataset = pd.read_csv("../../../Data_thesis/Full_Datasets/Full.csv")
dataset = dataset[["Date", "Hour", "Sensor", "SensorLongitude", "SensorLatitude", "CrowdednessCount"]]

In [46]:
dataset_dict = dataset.to_dict("index")

for k, v in tqdm(dataset_dict.items()):
    date = pd.to_datetime(v["Date"], format="%Y-%m-%d")
    hour = pd.to_datetime(v["Hour"]/100, unit="h")
    
    v['Date'] = pd.Timestamp.combine(date, hour.time())
    
dataset = pd.DataFrame.from_dict(dataset_dict, orient="index").drop(columns={"Hour"})

100%|████████████████████████████████████████████████████████████████████████| 103421/103421 [00:19<00:00, 5277.73it/s]


In [ ]:
for date in dataset["Date"].unique():
    temp = dataset[dataset["Date"] == date]
    
    for sensor in dataset["Sensor"].unique():
        
        if sensor not in temp["Sensor"]:
            lon = dataset[dataset["Sensor"] == sensor].reset_index()["SensorLongitude"][0]
            lat = dataset[dataset["Sensor"] == sensor].reset_index()["SensorLatitude"][0]
            
            dataset.append({"Date": date, "Sensor": sensor, "SensorLongitude": lon,
                           "SensorLatitude": lat, "CrowdednessCount": 0}, ignore_index=True) 

In [23]:
dataset.head()

,Date,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
0,2018-03-11 01:00:00,GAWW-04,4.897908,52.373283,886
1,2018-03-11 21:00:00,GAWW-07,4.900441,52.374414,1603
2,2018-03-11 21:00:00,GAWW-08,4.897193,52.371650,21
3,2018-03-11 21:00:00,GAWW-09,4.898479,52.375040,88
4,2018-03-11 21:00:00,GAWW-10,4.898808,52.372369,49


## TimeSeries Plot

In [ ]:
x_column = 'SensorLongitude'
y_column = 'SensorLatitude'
bubble_column = 'Sensor'
time_col = 'Date'
size_column = 'CrowdednessCount' 

grid = pd.DataFrame()

In [ ]:
# Make the grid
dates = dataset[time_col].unique()
col_name_template = '{}+{}_grid'
col_names = [x_column, y_column, bubble_column, size_column]
grid = make_grid(dataset, col_names, time_col, dates)
    
# Set the layout
figure, sliders_dict = set_layout(x_title='Sensor Longitude', y_title='Sensor Latitude', 
            title='Crowdedness Counts Amsterdam', x_logscale=True, y_logscale=False, 
            show_slider=True, slider_scale=dates, show_button=True, show_legend=True, 
            height=650)

# Add the base frame
date = min(dates)
col_name_template_date = col_name_template.format(date, {})
trace = get_trace(grid=grid, col_name_template=col_name_template_date, 
                    x_column=x_column, y_column=y_column, 
                    bubble_column=bubble_column, size_column=size_column)
figure['data'].append(trace)


# Add time frames
for date in tqdm(dates):
    frame = {'data': [], 'name': date}
    col_name_template_date = col_name_template.format(date, {})
    trace = get_trace(grid=grid, col_name_template=col_name_template_date, 
                    x_column=x_col, y_column=y_col, 
                    bubble_column=bubble_col, size_column=size_col)
    frame['data'].append(trace)
    figure['frames'].append(frame) 
    add_slider_steps(sliders_dict, date)

# Set the layout once more
figure['layout']['xaxis']['autorange'] = True
figure['layout']['yaxis']['range'] = [20, 100]
figure['layout']['sliders'] = [sliders_dict]

# Plot the animation
plotly.offline.plot(figure, config={'scrollzoom': True})